In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
df = pd.read_csv("recipes_full.csv")

In [3]:
df.head()

,id,title,description,instructions,ingredients
0,82,Grilled Chicken Breast,Simply grilled chicken with lemon and herbs.,1. Prepare ingredients. 2. Cook accordingly. 3...,"olive oil, chicken breast, lemon juice, thyme"
1,60,Kuru Fasulye,Classic Turkish white bean stew with tomato sa...,1. Soak beans overnight. 2. Sauté onions and t...,"white beans, onion, tomato paste, olive oil"
2,61,Tavuk Sote,Chicken sautéed with mixed vegetables.,1. Prepare ingredients. 2. Cook accordingly. 3...,"onion, garlic, olive oil, chicken breast"
3,62,Tavuk Şiş,Marinated chicken skewers grilled to perfection.,1. Prepare ingredients. 2. Cook accordingly. 3...,"olive oil, red pepper flakes, salt, chicken br..."
4,63,Adana Kebap,Spicy minced meat kebab.,1. Prepare ingredients. 2. Cook accordingly. 3...,"ground beef, onion, red pepper flakes, salt"


In [5]:
df.rename(columns={"name": "title"}, inplace=True)
df.isnull().sum()

id              0
title           0
description     0
instructions    0
ingredients     0
dtype: int64

In [6]:
df.dropna(subset=["title", "ingredients"], inplace=True)

In [7]:
df["ingredients_str"] = df["ingredients"].str.replace(",", " ")

In [8]:
title_vectorizer = TfidfVectorizer()
title_matrix = title_vectorizer.fit_transform(df["title"])

In [9]:
ingredient_vectorizer = TfidfVectorizer()
ingredient_matrix = ingredient_vectorizer.fit_transform(df["ingredients_str"])

In [30]:
def recommend(query_title, user_ingredients, df, 
              title_vectorizer, title_matrix,
              ingredient_vectorizer, ingredient_matrix, 
              alpha=0.5, top_n=5):
    
    # 1. Başlık vektörü ve skorları
    title_vec = title_vectorizer.transform([query_title])
    title_scores = cosine_similarity(title_vec, title_matrix).flatten()

    # 2. Malzeme vektörü ve skorları
    feature_names = set(ingredient_vectorizer.get_feature_names_out())
    common_ingredients = set(user_ingredients).intersection(feature_names)

    if not common_ingredients:
        return [{"error": "Hiçbir malzeme eşleşmedi."}]

    ing_text = " ".join(common_ingredients)
    ing_vec = ingredient_vectorizer.transform([ing_text])
    ing_scores = cosine_similarity(ing_vec, ingredient_matrix).flatten()

    combined_scores = alpha * title_scores + (1 - alpha) * ing_scores
    top_indices = combined_scores.argsort()[-top_n:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "title": df.iloc[idx]["title"],
            "ingredients": df.iloc[idx]["ingredients"],
            "score": round(combined_scores[idx], 4)  # daha hassas round
        })

    return results

In [51]:
recommend(
    query_title="Mercimek Çorbası",
    user_ingredients=["milk", "butter", "red lentils","onion"],
    df=df,
    title_vectorizer=title_vectorizer,
    title_matrix=title_matrix,
    ingredient_vectorizer=ingredient_vectorizer,
    ingredient_matrix=ingredient_matrix,
    alpha=0.5
)

[{'title': 'Mercimek Çorbası',
  'ingredients': 'red lentils, onion, milk, butter',
  'score': 0.887},
 {'title': 'Ezogelin Çorbası',
  'ingredients': 'onion, tomato paste, bulgur, red lentils',
  'score': 0.353},
 {'title': 'Mercimek Köftesi',
  'ingredients': 'onion, bulgur, red lentils, red pepper flakes',
  'score': 0.331},
 {'title': 'Pilav Üstü Et',
  'ingredients': 'onion, rice, butter, chicken breast',
  'score': 0.258},
 {'title': 'Hünkar Beğendi',
  'ingredients': 'eggplant, ground beef, butter, milk',
  'score': 0.258}]

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

title_vec = title_vectorizer.transform(["Zeytinyağlı Fasulye"])
title_scores = cosine_similarity(title_vec, title_matrix)

print("Başlık skoru:", max(title_scores[0]))  # muhtemelen 1.0 değil, mesela 0.98

user_ing_text = " ".join(["onion", "tomato", "olive oil", "white beans"])
ing_vec = ingredient_vectorizer.transform([user_ing_text])
ing_scores = cosine_similarity(ing_vec, ingredient_matrix)

print("Malzeme skoru:", max(ing_scores[0]))  # yine 1.0 olmayabilir, mesela 0.96

Başlık skoru: 1.0
Malzeme skoru: 1.0


In [41]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
df = pd.read_csv("recipes_full.csv")

# Eğitim
title_vectorizer = TfidfVectorizer()
title_matrix = title_vectorizer.fit_transform(df["title"])

ingredient_vectorizer = CountVectorizer()
ingredient_matrix = ingredient_vectorizer.fit_transform(df["ingredients"])


In [43]:
def recommend(query_title, user_ingredients, df, 
              title_vectorizer, title_matrix,
              ingredient_vectorizer, ingredient_matrix,
              alpha=0.5, top_n=5):
    
    # Başlık skoru
    title_vec = title_vectorizer.transform([query_title])
    title_scores = cosine_similarity(title_vec, title_matrix).flatten()

    # Malzeme skoru
    all_features = set(ingredient_vectorizer.get_feature_names_out())
    user_ing = set(user_ingredients).intersection(all_features)
    if not user_ing:
        return "Hiçbir malzeme eşleşmedi."

    user_ing_text = " ".join(user_ing)
    ing_vec = ingredient_vectorizer.transform([user_ing_text])
    ing_scores = cosine_similarity(ing_vec, ingredient_matrix).flatten()

    # Skorları karıştır
    combined = alpha * title_scores + (1 - alpha) * ing_scores
    indices = combined.argsort()[-top_n:][::-1]

    results = []
    for idx in indices:
        results.append({
            "title": df.iloc[idx]["title"],
            "ingredients": df.iloc[idx]["ingredients"],
            "score": round(combined[idx], 3)
        })
    return results


In [45]:
recommend(
    query_title="Mercimek Çorbası",
    user_ingredients=["milk", "butter", "red lentils", "onion"],
    df=df,
    title_vectorizer=title_vectorizer,
    title_matrix=title_matrix,
    ingredient_vectorizer=ingredient_vectorizer,
    ingredient_matrix=ingredient_matrix,
    alpha=0.5
)


[{'title': 'Mercimek Çorbası',
  'ingredients': 'red lentils, onion, milk, butter',
  'score': 0.887},
 {'title': 'Ezogelin Çorbası',
  'ingredients': 'onion, tomato paste, bulgur, red lentils',
  'score': 0.353},
 {'title': 'Mercimek Köftesi',
  'ingredients': 'onion, bulgur, red lentils, red pepper flakes',
  'score': 0.331},
 {'title': 'Pilav Üstü Et',
  'ingredients': 'onion, rice, butter, chicken breast',
  'score': 0.258},
 {'title': 'Hünkar Beğendi',
  'ingredients': 'eggplant, ground beef, butter, milk',
  'score': 0.258}]

In [52]:
with open("recipe_model.pkl", "wb") as f:
    pickle.dump({
        "df": df,
        "title_vectorizer": title_vectorizer,
        "title_matrix": title_matrix,
        "ingredient_vectorizer": ingredient_vectorizer,
        "ingredient_matrix": ingredient_matrix
    }, f)